In [1]:
import pandas as pd
import pydeck as pdk
import numpy as np


In [50]:
input_filename = 'output_H3binned_data/OBIS_h3counts_res6.csv'
output_base = 'output_H3binned_data/html_visualizations/' + input_filename.replace('.csv', '').replace('output_H3binned_data/', '')
output_base

'output_H3binned_data/html_visualizations/OBIS_h3counts_res6'

In [51]:
df_hex = pd.read_csv(input_filename)

In [52]:
df_hex

,h3_index,count,count_private,classification,latitude,longitude
0,86000005fffffff,1,0,ocean,79.41009,37.73405
1,8600000cfffffff,2,0,ocean,79.43435,38.25291
2,8600000dfffffff,1,0,ocean,79.41808,38.56040
3,860000127ffffff,1,0,ocean,79.06628,37.50484
4,8600001afffffff,1,0,ocean,79.13057,38.21124
...,...,...,...,...,...,...
1664652,86f3b506fffffff,1,0,ocean,-78.82663,166.32705
1664653,86f3b5397ffffff,1,0,ocean,-79.00151,165.89472
1664654,86f3b5437ffffff,1,0,ocean,-78.88966,163.17447
1664655,86f3b5807ffffff,2,0,coast,-78.44380,165.68916


In [53]:
df_plot_all_obis = df_hex[df_hex['classification'] != 'land']
# Drop the 'count' column
df_plot_all_obis = df_plot_all_obis.drop(columns=['count_private', 'latitude', 'longitude' ,'classification'])
df_plot_all_obis

,h3_index,count
0,86000005fffffff,1
1,8600000cfffffff,2
2,8600000dfffffff,1
3,860000127ffffff,1
4,8600001afffffff,1
...,...,...
1664647,86f3b068fffffff,2
1664652,86f3b506fffffff,1
1664653,86f3b5397ffffff,1
1664654,86f3b5437ffffff,1


In [54]:
# Apply the filter to exclude 'land' classification
df_plot_private_only = df_hex[df_hex['classification'] != 'land']
# Drop rows where count_private is 0
df_plot_private_only = df_plot_private_only[df_plot_private_only['count_private'] != 0]
# Drop the 'count' column
df_plot_private_only = df_plot_private_only.drop(columns=['count', 'latitude', 'longitude','classification'])
# Rename 'count_private' to 'count'
df_plot_private_only = df_plot_private_only.rename(columns={'count_private': 'count'})
df_plot_private_only

,h3_index,count
351,86000c2dfffffff,2
409,860010427ffffff,1
512,86001aa37ffffff,63
1713,86002c927ffffff,2
1723,86002cb8fffffff,1
...,...,...
1664637,86f3adb2fffffff,4
1664638,86f3adb37ffffff,7
1664639,86f3adb67ffffff,3
1664640,86f3adb77ffffff,6


In [55]:
# Define the colors for minimum, midpoint (15%), and maximum
low_color = [32, 10, 58, 150]  # Fully transparent at min
mid_color = [255, 216, 11, 200]  # Midpoint color (15%)
high_color = [255, 253, 240, 255]  # Max color

def calculate_color(count, min_count, max_count, diff, low_color, high_color, mid_color, clip_percent):
    # Apply clipping
    normalized_count = (count - min_count) / diff
    
    # Clip values at the top end
    clip_value = 1.0 - (clip_percent / 100.0)
    clipped_count = np.clip(normalized_count, 0.0, clip_value)
    
    # Rescale clipped values so that the maximum clipped value maps to 1.0
    rescaled_count = clipped_count / clip_value
    
    # Calculate the RGBA values
    red = np.interp(rescaled_count, [0.0, 0.05, 1.0], [low_color[0], mid_color[0], high_color[0]])
    green = np.interp(rescaled_count, [0.0, 0.05, 1.0], [low_color[1], mid_color[1], high_color[1]])
    blue = np.interp(rescaled_count, [0.0, 0.05, 1.0], [low_color[2], mid_color[2], high_color[2]])
    alpha = np.interp(rescaled_count, [0.0, 0.05, 1.0], [low_color[3], mid_color[3], high_color[3]])
    
    return [int(red), int(green), int(blue), int(alpha)]

def create_color_scheme(df_reference, df_apply, clip_percent=0):
    min_count = df_reference['count'].min()
    max_count = df_reference['count'].max()
    diff = max_count - min_count
    clip_value = 1.0 - (clip_percent / 100.0)
    
    # Calculate the exact count values for the color scale
    high_count = min_count + clip_value * diff
    mid_count = min_count + (0.15 * (high_count- min_count))

    # Generate color scheme for each count value in df_apply
    color_scheme = df_apply['count'].apply(
        lambda count: calculate_color(count, min_count, max_count, diff, low_color, high_color, mid_color, clip_percent)
    )
    
    return color_scheme, min_count, mid_count, high_count

In [57]:
# tested 70 for res2 and 98 for res4 , 99.8 for res6
clip_percent = 99.8

In [58]:
# Apply the function
df_plot=df_plot_all_obis.copy()
df_plot['color_scheme'], min_count, mid_count, high_count = create_color_scheme(df_plot_all_obis, df_plot_all_obis, clip_percent=clip_percent)
output_html = f"{output_base}_alldata.html"

print(f"plotting the hex counts for ALL of OBIS. HTML version written here: {output_html}")

plotting the hex counts for ALL of OBIS. HTML version written here: output_H3binned_data/html_visualizations/OBIS_h3counts_res6_alldata.html


In [64]:
# Apply the function
df_plot=df_plot_private_only.copy()
df_plot['color_scheme'], min_count, mid_count, high_count = create_color_scheme(df_plot_all_obis, df_plot_private_only, clip_percent=clip_percent)
output_html = f"{output_base}_privateonly.html"

print(f"plotting the hex counts for PRIVATE ONLY occurences of OBIS. HTML version written here: {output_html}")

plotting the hex counts for PRIVATE ONLY occurences of OBIS. HTML version written here: output_H3binned_data/html_visualizations/OBIS_h3counts_res6_privateonly.html


In [65]:
print(min_count, mid_count, high_count)

1 768.8575000000006 5120.050000000005


In [66]:
df_hex.tail(2)

,h3_index,count,count_private,classification,latitude,longitude
1664655,86f3b5807ffffff,2,0,coast,-78.44380,165.68916
1664656,86f3b5857ffffff,1,0,land,-78.50676,166.12610


In [67]:
# Create the H3HexagonLayer
layer = pdk.Layer(
    'H3HexagonLayer',
    df_plot,
    get_hexagon='h3_index',        # H3 index field in your data
    get_fill_color='color_scheme',
    opacity=1,
    extruded=False,
    get_line_color=[32, 10, 58, 10],
    line_width_min_pixels=1,
    pickable=True,
    filled=True,
    stroked=True
)

# Set the initial view state
view = pdk.ViewState(
    latitude=20.0,  # Center latitude
    longitude=10.0, # Center longitude
    zoom=0.5,
    pitch=0
)

# Create a Pydeck Deck instance
deck = pdk.Deck(
    layers=[layer],
    initial_view_state=view,
    tooltip={"text": "{count}"},
    map_provider="mapbox", 
    map_style='mapbox://styles/oceandatafoundation/clwg6xklg00an01pcgmeufjxq', 
    api_keys={'mapbox': 'pk.eyJ1Ijoib2NlYW5kYXRhZm91bmRhdGlvbiIsImEiOiJjazk5bGxpNWkwYWU1M2Vya3hkcHh4czdrIn0.yf7kIiPfDNE7KP9_9wTN6A'}
)

# Display the map
deck


In [68]:
deck.to_html(output_html, notebook_display=False)


In [69]:
# Read the generated HTML file
with open(output_html, 'r') as file:
    html_content = file.read()

# Add custom HTML and CSS for the color scale with counts
color_scale_html = f"""
<style>
    body {{
        margin: 0;
        padding: 0;
    }}
    .color-scale {{
        position: absolute;
        bottom: 10px;
        left: 10px;
        background: white;
        padding: 10px;
        border-radius: 5px;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        z-index: 1000; /* Ensure the color scale is on top */
    }}
    .color-scale div {{
        display: flex;
        align-items: center;
        margin-bottom: 5px;
    }}
    .color-scale div:last-child {{
        margin-bottom: 0;
    }}
    .color-box {{
        width: 20px;
        height: 20px;
        margin-right: 10px;
    }}
</style>
<div class="color-scale">
    <div><div class="color-box" style="background-color: rgba({low_color[0]}, {low_color[1]}, {low_color[2]}, {low_color[3]/255});"></div>Low ({int(min_count)})</div>
    <div><div class="color-box" style="background-color: rgba({mid_color[0]}, {mid_color[1]}, {mid_color[2]}, {mid_color[3]/255});"></div>Mid ({int(mid_count)})</div>
    <div><div class="color-box" style="background-color: rgba({high_color[0]}, {high_color[1]}, {high_color[2]}, {high_color[3]/255});"></div>High ({int(high_count)})</div>
</div>
"""

# Insert the color scale HTML before the closing </body> tag
html_content = html_content.replace('</body>', color_scale_html + '</body>')

# Write the modified HTML content to a new file
with open(output_html, 'w') as file:
    file.write(html_content)

print("HTML file with updated color scale created")

HTML file with updated color scale created
